In [1]:
from core.utils import Tibanna
from core import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
    

In [3]:
import time
from tasks import run_fastqc
from invoke import run

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

add_wfr = False

# url for hic exps
fastq_url = "search/?type=FileFastq&quality_metric.uuid=No+value" + \
            "&status=released&status=released%20to%20project&status=uploaded" + \
            "&limit=all" + \
            "&q=date_created%3A%3E%3D2016-09-01"

fastq_files = ff_utils.get_metadata(fastq_url , connection=ff)['@graph']


counter = 0
completed = 0
print len(fastq_files)

okay = 0
non = 0
running = 0
for a_fastq in fastq_files:
    report = get_wfr_out(a_fastq['accession'], 'fastqc-0-11-4-1/1', None, ff)
    if report.startswith('no') or not report:
        non += 1
        if add_wfr:
            print 'fastqc running for', a_fastq['accession']
            code_qc= "invoke run_fastqc " + env + " " + a_fastq['display_title'] + " " + a_fastq['uuid']
            run(code_qc)
        else:
            print 'fastqc run missing for', a_fastq['accession']   
    elif report == 'running':
        running += 1
        print(a_fastq['accession'], 'still running')
    else:
        okay += 1
        
print("okay, not okay, running")    
print(okay, non, running)


0
okay, not okay, running
(0, 0, 0)


In [5]:
# Release fastq qc

fastq_url = '/search/?status=released&status=released%20to%20project&status=archived&type=FileFastq'
fastq_files = ff_utils.get_metadata(fastq_url , connection=ff)['@graph']
fastq_files = [i for i in fastq_files if i.get('quality_metric')]
print len(fastq_files)
success = 0
patch_problem = 0

patch_trigger = True

for fastq in fastq_files:
    fastq_status = fastq['status']
    if fastq_status in ['released', 'released to project', 'archived']:
        qc_status = ff_utils.get_metadata(fastq['quality_metric'],connection=ff)['status']
        if qc_status != fastq_status:
            print('unmatched status', fastq['accession'])
            patch_data = {"status": fastq_status}
            
            if patch_trigger:
                try:
                    ff_utils.patch_metadata(patch_data, obj_id=fastq['quality_metric'] ,connection=ff)
                    success += 1
                    print success
                except Exception as e:
                    print e
                    print
                    patch_problem += 1
print success
print patch_problem



2743
('unmatched status', u'4DNFIHZ4WNM6')
1
('unmatched status', u'4DNFIC5N164E')
2
('unmatched status', u'4DNFIRZ68YAB')
3
('unmatched status', u'4DNFIAZ92Z7Y')
4
('unmatched status', u'4DNFIEW8ZI49')
5
('unmatched status', u'4DNFIEURG18Y')
6
('unmatched status', u'4DNFIF61NGMY')
7
('unmatched status', u'4DNFIK1RIZNE')
8
('unmatched status', u'4DNFIAA48CH2')
9
('unmatched status', u'4DNFIDOS8NCD')
10
('unmatched status', u'4DNFISRKPRA8')
11
('unmatched status', u'4DNFIU3EWJGB')
12
('unmatched status', u'4DNFILVLKLHD')
13
('unmatched status', u'4DNFIX8IVO3W')
14
('unmatched status', u'4DNFIQT9PZC1')
15
('unmatched status', u'4DNFI86VEI2U')
16
('unmatched status', u'4DNFI6MELUQP')
17
('unmatched status', u'4DNFIG357V4G')
18
('unmatched status', u'4DNFIMZQGJLU')
19
('unmatched status', u'4DNFIPSUUBFC')
20
('unmatched status', u'4DNFI9LFKVBR')
21
('unmatched status', u'4DNFINEOCYA5')
22
('unmatched status', u'4DNFICK3J4FU')
23
('unmatched status', u'4DNFIU2JFFBP')
24
('unmatched status',